### Issue with Gradio when using Claude for Conversational AI (chatbots)

As explained in Day 3 (notebook), Gradio has been upgraded to pass in history in a format OpenAI accepts.

This update simplifies the development work as Gradio manages the history in the background and provides a history structure that matches OpenAi. Fortunately, this works with other models that leverage the client libraries for OpenAI, such as Llama and Gemini. 

However, leveraging Gradio's ChatInterface while using Claude models generates a BadRequest error. 

In analyzing the history list from Gradio, it has the following  format:

`{'role': 'assistant', 'metadata': None, 'content': '[assistant message here]', 'options': None}`

OpenAi accepts this format without issues, as do other models such as Llama and Gemini - at least while leveraging the client libraries for OpenAI. They accept both formats. 

However, Claude's API requires the following format:

`{'role': 'user', 'content': '[user message here]'}`

Claude rejects anything different from this format.

Run the code below to get the details! 

In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr
from pprint import pprint # for a nicely formatted printout of a list

In [0]:
# set environment variables for OpenAi
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

# validate API Key
if not openai_api_key:
    raise ValueError("No OpenAI API key was found! Please check the .env file.")
else:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")

if not anthropic_api_key:
    raise ValueError("No Anthropic API key was found! Please check the .env file.")
else:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")

if not google_api_key:
    raise ValueError("No Gemini API key was found! Please check the .env file.")
else:
    print(f"Gemini API Key exists and begins {google_api_key[:8]}")

In [0]:
# constants
MODELS = { 'GPT': 'gpt-4o-mini', 
          'LLAMA': 'llama3.2', 
          # 'DEEPSEEK': 'deepseek-r1:1.5b',
          'CLAUDE': 'claude-3-haiku-20240307',
          'GEMINI': 'gemini-2.0-flash-exp'
         }

CLIENTS = { 'GPT': OpenAI(), 
            'LLAMA': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
            # 'DEEPSEEK': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
            'CLAUDE': anthropic.Anthropic(),
            'GEMINI': OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=google_api_key)
          }

# system prompt
system_message = "You are a nice assistant that like to chat with users"

# to save/print the history structure
console = []

#### Testing GPT

This runs without issues. You may change the model to Llama or Gemini for further testing.

In [0]:
def gpt_chat(message, history):
    # change to Llama or Gemini for test
    model = 'GPT'

    # add system message to history
    if not history:
        history.append({"role": "system", "content": system_message})

    # conversation including new user message
    messages = history + [{"role": "user", "content": message}]

    # send request to OpenAi
    response = CLIENTS[model].chat.completions.create(
                model = MODELS[model],
                max_tokens = 200,
                messages = messages,
            )

    # save history structure
    global console
    console = history[:]
    
    # post in Gradio's chat interface
    return response.choices[0].message.content

##### Have a conversation of several messages (3 or more)

In [0]:
# Gradio with GPT
gr.ChatInterface(fn=gpt_chat, type="messages", examples=["How are you today?", "Please, tell me a joke."]).launch()

##### Notice how the history structure includes both formats, and this is okay with OpenAi

In [0]:
# audit console
pprint(console)

# empty list
console.clear()

#### Testing with Claude

This first test will generate a BadRequest error on the second message from the user. The first message sent follows the required format preventing errors.

In [0]:
# Chat using Gradio's history - generates error after the second message from user
def claude_chat(message, history): # Gradio requires the history parameter, but it goes unused. 
    
    # save history structure
    global console
    console = history[:]
    
    model = 'CLAUDE'

     # conversation including new user message - this is why the first message does not generate an error
    messages = history + [{"role": "user", "content": message}]

    # send the request to Claude
    response = CLIENTS[model].messages.create(
        model = MODELS[model],
        max_tokens = 200,
        system = system_message,
        messages = messages,
    )
    
    # post in Gradio's chat interface
    return response.content[0].text

##### Have a conversation of several messages (3 or more)

In [0]:
# Gradio with Claude
gr.ChatInterface(fn=claude_chat, type="messages", examples=["How are you today?", "Please, tell me a joke."]).launch()

##### Notice how the history structure changes for the second message. This cause a BadRequest error with Claude.

In [0]:
# audit console
pprint(console)

# empty list
console.clear()

##### Have a new conversation of several messages (3 or more), but this time leveraging a local history repository instead of Gradio's. 

The code leverages the list `console` as the local repository. Note that Gradio still requires the second parameter (history) is required even though it is not used. 

In [0]:
def claude_chat(message, history): # Gradio requires the history parameter, but it goes unused. 
    # local history repository instead of Gradio's 
    global console
    model = 'CLAUDE'

    # append new user message to history - using Claude's required format
    console.append({"role": "user", "content": message})

    # send the request to Claude
    response = CLIENTS[model].messages.create(
        model = MODELS[model],
        max_tokens = 200,
        system = system_message,
        messages = console, # use local history repository
    )

    # append the assistant response to history - using Claude's required format
    console.append({"role": "assistant", "content": response.content[0].text})

    # post in Gradio's chat interface
    return response.content[0].text

In [0]:
# Gradio with Claude
gr.ChatInterface(fn=claude_chat, type="messages", examples=["How are you today?", "Please, tell me a joke."]).launch()

##### Notice that the history structure follows the required format.

In [0]:
# audit console
pprint(console)

# empty list
console.clear()

### Conclusion

1. OpenAi (and models that leverage the client libraries for OpenAI) supports both formats for the conversation history.

    (Gradio's) `{'role': 'assistant', 'metadata': None, 'content': '[assistant message here]', 'options': None}`

    (Claude's) `{'role': 'user', 'content': '[user message here]'}`
   
2. Claude only supports the following format for the conversation history:

    (Claude's) `{'role': 'user', 'content': '[user message here]'}`